In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [ ]:
df_madrid = pd.read_csv("madrid.csv")
df_rome = pd.read_csv("rome.csv")
df_paris = pd.read_csv("paris.csv")
df_oslo = pd.read_csv("oslo.csv")
df_london = pd.read_csv("london.csv")
df_geneva = pd.read_csv("geneva.csv")
df_dublin = pd.read_csv("dublin.csv")
df_barcelona = pd.read_csv("barcelona.csv")
df_athens = pd.read_csv("athens.csv")
df_amsterdam = pd.read_csv("amsterdam.csv")


l_dataframes = [df_madrid, df_barcelona, df_rome, df_paris, df_oslo, df_london, df_geneva, df_dublin, df_athens, df_amsterdam]

In [ ]:
for i in l_dataframes:
    i.drop("source", axis = 1, inplace = True)

In [ ]:
df = pd.concat(l_dataframes)
df

In [ ]:
l_columns_chosen = ['id', 'neighbourhood_cleansed',
                    'room_type', 'accommodates',
                    'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
                    'minimum_nights', 'maximum_nights', 'availability_365',
                    'number_of_reviews', 'reviews_per_month', 'host_total_listings_count']
df = df[l_columns_chosen]

In [ ]:
df.isna().sum()/df.shape[0]*100

In [ ]:
df["bathrooms_text"] = df["bathrooms_text"].apply(lambda x : x.split(" ")[0] if (type(x) == str) and ("bath" in x) else np.nan)

bath_map = {"Private" : 1, "Shared" : 0.5, "Half-bath" : 0.5}

df["bathrooms_text"] = df["bathrooms_text"].apply(lambda x : bath_map[x] if x in bath_map.keys() else float(x))

In [ ]:
# amenities: número de elementos en la lista

list_amenities = list()

for num, x in enumerate(df["amenities"].values):
    try:
        list_amenities.append(len(json.loads((x.replace("'", '"')))))
        
    except:
        list_amenities.append(len(x.replace("'", '"').replace("\\", "")))
        
        
df["amenities"] = list_amenities

In [ ]:
%%time

import json
from collections import Counter
l_amenities_cleaned, l_amenities_sublisted = list(),list()
for i in df["amenities"]:
    
    l_amenities_sublisted.append(json.loads(i))
    
    for j in json.loads(i):
    
        l_amenities_cleaned.append(j)

l_values = list(Counter(l_amenities_cleaned).values())
l_keys = list(Counter(l_amenities_cleaned).keys())

l_valuable_amenities, l_not_relevant_amenities = [], []

for i in l_values:
    
    if i > 10000:
        
        l_valuable_amenities.append(l_keys[l_values.index(i)])
        
    else:
        
        l_not_relevant_amenities.append(l_keys[l_values.index(i)])
        
cont = 0
l_popular, l_exclusive = [],[]

for i in l_amenities_sublisted:
    
    for j in l_valuable_amenities:
        
        if j in i:
            
            cont+=1
        
    if cont > len(i)/2:

        l_popular.append(1)

    else:

        l_popular.append(0)

    cont = 0
        
    for k in l_not_relevant_amenities:
        
        if k in i:
            
            cont += 1
            
    if cont > len(i)/2:

        l_exclusive.append(1)

    else:

        l_exclusive.append(0)
            
    cont = 0
    
df["popular_amenities"] = l_popular
df["exclusive_amenities"] = l_exclusive

In [ ]:
%%time

import json
from collections import Counter
l_amenities_cleaned, l_amenities_sublisted = list(),list()
for i in df["amenities"]:
    
    l_amenities_sublisted.append(json.loads(i))
    
    for j in json.loads(i):
    
        l_amenities_cleaned.append(j)

In [ ]:
%%time

l_values = list(Counter(l_amenities_cleaned).values())
l_keys = list(Counter(l_amenities_cleaned).keys())

l_valuable_amenities, l_not_relevant_amenities = [], []

for i in l_values:
    
    if i > 10000:
        
        l_valuable_amenities.append(l_keys[l_values.index(i)])
        
    else:
        
        l_not_relevant_amenities.append(l_keys[l_values.index(i)])

In [ ]:
cont = 0
l_popular, l_exclusive = [],[]

for i in l_amenities_sublisted:
    
    for j in l_valuable_amenities:
        
        if j in i:
            
            cont+=1
        
    if cont > len(i)/2:

        l_popular.append(1)

    else:

        l_popular.append(0)

    cont = 0
        
    for k in l_not_relevant_amenities:
        
        if k in i:
            
            cont += 1
            
    if cont > len(i)/2:

        l_exclusive.append(1)

    else:

        l_exclusive.append(0)
            
    cont = 0
    
df["popular_amenities"] = l_popular
df["exclusive_amenities"] = l_exclusive

In [ ]:
#df.drop("amenities", axis = 1, inplace=True)

In [ ]:
df.dropna(inplace = True)

In [ ]:
df["price"] = df["price"].apply(lambda x: float(x.strip("$").replace(',', '')) if pd.notnull(x) else x).values

In [ ]:
df

In [ ]:
df_num = df._get_numeric_data()

In [ ]:
df_cat = df.drop(df_num.columns, axis = 1)
df_cat = pd.concat([df_cat, df_num[["id"]]], axis = 1)

In [ ]:
# room_type: eliminar "Hotel room" y transformar a dummies

df_cat = df_cat[df_cat["room_type"] != "Hotel room"]

df_cat = pd.concat([df_cat, pd.get_dummies(data = df_cat["room_type"])], axis = 1).drop("room_type", axis = 1)

In [ ]:
# Uso el LabelEncoder para transformarlo a números sin ningún tipo de relación u orden

from sklearn.preprocessing import LabelEncoder

neighbourhood_encoder = LabelEncoder()
df_cat["neighbourhood_cleansed"] = neighbourhood_encoder.fit_transform(df_cat["neighbourhood_cleansed"])

In [ ]:
# Uso el LabelEncoder para transformarlo a números sin ningún tipo de relación u orden

from sklearn.preprocessing import LabelEncoder

city_encoder = LabelEncoder()
df_cat["city"] = city_encoder.fit_transform(df_cat["city"])

In [ ]:
df = pd.merge(left = df_num, right = df_cat, on = "id")

df

### Outliers

In [ ]:
df["bathrooms_text"]

In [ ]:
df["bathrooms_text"] = df["bathrooms_text"].apply(lambda x : np.ceil(x))

In [ ]:
for i in df.columns:
    
    print(i)
    sns.kdeplot(df[i])
    plt.show()

In [ ]:
df = df[df["accommodates"] <= 8]
df = df[df["bathrooms_text"].between(1, 2)]
df = df[df["bedrooms"] <= 4]
df = df[df["beds"].between(1, 5)]
df = df[df["amenities"] <= 50]
df = df[df["minimum_nights"] <= 30]
df = df[df["number_of_reviews"] <= 300]
df = df[df["reviews_per_month"] <= 8]
df = df[df["price"] <= 400]

In [ ]:
df.drop("Shared room", axis = 1, inplace = True)

In [ ]:
# for i in df.columns:
    
#     print(i)
#     sns.kdeplot(df[i])
#     plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

x_scaler = MinMaxScaler()
df[df.drop("price", axis = 1).columns] = x_scaler.fit_transform(df[df.drop("price", axis = 1).columns])

y_scaler = MinMaxScaler()
df["price"] = y_scaler.fit_transform(df[["price"]]).flatten()

In [ ]:
df.drop(["id"], axis = 1, inplace = True)

In [ ]:
df.columns

In [ ]:
X = df.drop(["price"], axis = 1)
y = df["price"]

print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(f"X_train: {X_train.shape} | y_train: {y_train.shape}")
print(f"X_test: {X_test.shape} | y_test: {y_test.shape}")

In [ ]:
# Modelos
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Métricas
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
%%time

modelos = [LinearRegression(), KNeighborsRegressor(), DecisionTreeRegressor(),
           RandomForestRegressor(), SVR(), AdaBoostRegressor(), GradientBoostingRegressor()]

metricas = list() 

for model in modelos:
    
    # fit
    model.fit(X_train, y_train)
    
    # predict
    yhat = model.predict(X_test)
    
    # metrics
    r2 = r2_score(y_test, yhat)
    mse = mean_squared_error(y_test, yhat)
    
    metricas.append([str(model), r2, mse, model])
    
df_metricas = pd.DataFrame(data = metricas, columns = ["model_name", "r2", "mse", "model"])

df_metricas.sort_values(by = "r2", ascending = False)

In [ ]:
modelo = df_metricas["model"][3]

modelo

In [ ]:
yhat = modelo.predict(X_test)

for i, j in zip(yhat[:5], y_test[:5]):
    print(f"Predicción:{i} \tValor real:{j}")

In [ ]:
# Distance btw real and predicted values

plt.figure(figsize = (8, 5))

sns.scatterplot(x = y_test, y = yhat, alpha = 0.5, color = "blue")

plt.xlabel("Valores Reales (y_train)", size = 18)
plt.ylabel("Predicciones (yhat)", size = 18)

plt.show()

In [ ]:
importances = np.argsort(modelo.feature_importances_)[::-1]

for i in importances:
    
    print(i, modelo.feature_importances_[i]*100, df.drop("price", axis = 1).columns[i])

In [ ]:
y_test_inv = y_scaler.inverse_transform([y_test]).flatten()
yhat_inv = y_scaler.inverse_transform([yhat]).flatten()


r2 = r2_score(y_test_inv, yhat_inv)
mse = mean_squared_error(y_test_inv, yhat_inv)

print(r2, mse)

# GridSearch

In [ ]:
# %%time

# from sklearn.model_selection import GridSearchCV

# # Modelo
# model = RandomForestRegressor()

# # Parametros a iterar
# params = {"n_estimators" : [i for i in range(100, 1001, 50)],
#           "max_depth"    : [8, 10, 12, 14, 16],
#           "max_features" : ["log2", "sqrt"]}

# # Metricas
# scorers = {"r2", "neg_mean_squared_error"}

# #GridSearchCV
# grid_solver = GridSearchCV(estimator  = model, 
#                            param_grid = params, 
#                            scoring    = scorers,
#                            cv         = 10,
#                            refit      = "r2",
#                            n_jobs     = -1, 
#                            verbose    = 2)

# # Resultados
# model_result = grid_solver.fit(X_train, y_train)

In [ ]:
# model_result.best_estimator_

In [ ]:
# model_result.cv_results_["mean_test_r2"].max()

In [ ]:
# model_result.cv_results_

In [ ]:
# model_result.best_score_

# Training the definitive model

In [ ]:
# X_def = df.drop(["price"], axis = 1)
# y_def = df["price"]

In [ ]:
# model = RandomForestRegressor(max_depth=16, max_features='sqrt', n_estimators=650, random_state = 42)
# model.fit(X_def, y_def)

## Downloading the model

In [ ]:
# import pickle

# with open("model_rf.sav", "wb") as file:
#     pickle.dump(model, file)